In [1]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
print("Num GPUs Available: ", len(physical_devices))
if physical_devices:
    try:
        for gpu in physical_devices:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

Num GPUs Available:  1


In [34]:
import torch
from transformers import TFLongformerModel, LongformerTokenizerFast, LongformerConfig
import tensorflow as tf

config = LongformerConfig.from_pretrained('allenai/longformer-base-4096')
# choose the attention mode 'n2', 'tvm' or 'sliding_chunks'
# 'n2': for regular n2 attantion
# 'tvm': a custom CUDA kernel implementation of our sliding window attention
# 'sliding_chunks': a PyTorch implementation of our sliding window attention
config.attention_mode = 'sliding_chunks'

model = TFLongformerModel.from_pretrained('allenai/longformer-base-4096', config = config)
tokenizer = LongformerTokenizerFast.from_pretrained('allenai/longformer-base-4096')
tokenizer.model_max_length = model.config.max_position_embeddings

def LFencode(row):
    SAMPLE_TEXT = row['selftext']
    input_ids = tf.expand_dims(tf.convert_to_tensor(tokenizer.encode(SAMPLE_TEXT)), 0) # batch of size 1

    # model = model.cuda(); input_ids = input_ids.cuda()

    # Attention mask values -- 0: no attention, 1: local attention, 2: global attention
    attention_mask = tf.ones(input_ids.shape, dtype=tf.int32) # initialize to local attention

    outputs = model(input_ids, attention_mask=attention_mask)
    pooled_output = outputs.pooler_output
    return pooled_output

Some layers from the model checkpoint at allenai/longformer-base-4096 were not used when initializing TFLongformerModel: ['lm_head']
- This IS expected if you are initializing TFLongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFLongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFLongformerModel were initialized from the model checkpoint at allenai/longformer-base-4096.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFLongformerModel for predictions without further training.


In [35]:
input_ids.shape

TensorShape([1, 4002])

In [36]:
import pandas as pd
import numpy as np
nosleepDf = pd.read_csv('Download/Cleaned Data/NoSleep.csv')
selfDf = pd.read_csv('Download/Cleaned Data/Self.csv')

In [39]:
nosleepDfShort = nosleepDf[:2000].copy()
selfDfShort = selfDf[:2000].copy()

In [40]:
from tqdm.notebook import tqdm
tqdm.pandas()

selfDfShort['LF pooler output']= selfDfShort.progress_apply(LFencode, axis=1)
nosleepDfShort['LF pooler output']= nosleepDfShort.progress_apply(LFencode, axis=1)

selfDfShort.to_pickle('Download/Cleaned Data with Longformer/selfDfShort.pkl')
nosleepDfShort.to_pickle('Download/Cleaned Data with Longformer/nosleepDfShort.pkl')

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

In [3]:
import pandas as pd


nosleepDfShort = pd.read_pickle('Download/Cleaned Data with Longformer/nosleepDfShort.pkl')
selfDfShort = pd.read_pickle('Download/Cleaned Data with Longformer/selfDfShort.pkl')



In [4]:
nosleepLF = tf.stack(nosleepDfShort.loc[:,'LF pooler output'].to_list())
selfLF = tf.stack(selfDfShort.loc[:,'LF pooler output'].to_list())

nosleepLF = tf.reshape(nosleepLF, nosleepLF.shape[::2])
selfLF = tf.reshape(selfLF, selfLF.shape[::2])

In [5]:
import numpy as np
from sklearn.manifold import TSNE
import seaborn as sns

%matplotlib widget
import matplotlib.pyplot as plt

X_embedded_nosleep = TSNE(learning_rate=80, n_components=3).fit_transform(nosleepLF)
X_embedded_self = TSNE(learning_rate=80, n_components=3).fit_transform(selfLF)

df1 = pd.DataFrame(X_embedded_nosleep, columns=['x','y', 'z'])
df1['subreddit'] = 'nosleep'
df2 = pd.DataFrame(X_embedded_self, columns=['x','y', 'z'])
df2['subreddit'] = 'self'

df = pd.concat([df1,df2])

In [6]:
df

,x,y,z,subreddit
0,-38.624775,-2.294882,20.955492,nosleep
1,0.792412,-15.289292,14.973711,nosleep
2,40.421726,9.070208,7.038894,nosleep
3,-40.429996,-4.475523,14.154373,nosleep
4,0.247477,-15.553728,0.569228,nosleep
...,...,...,...,...
1995,22.816322,-13.808095,1.618495,self
1996,-10.423856,22.686611,-3.754926,self
1997,21.218271,17.061604,19.053509,self
1998,-9.011683,2.076463,-19.166342,self


In [7]:
from mpl_toolkits.mplot3d import Axes3D,axes3d
sns.set(style = "darkgrid")



fig = plt.figure()
fig.set_size_inches(18.5, 10.5)
ax = fig.add_subplot(111, projection = '3d')

ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")

ax.scatter(df[df['subreddit']=='nosleep']['x'], df[df['subreddit']=='nosleep']['y'], df[df['subreddit']=='nosleep']['z'], c='red', label='nosleep')
ax.scatter(df[df['subreddit']=='self']['x'], df[df['subreddit']=='self']['y'], df[df['subreddit']=='self']['z'], c='blue', label='self')
plt.legend()

plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [132]:
X, Y, Z = axes3d.get_test_data(0.1)


In [134]:
df[df['subreddit']=='nosleep']['z']

0      -20.150057
1      -12.655766
2       -8.502742
3      -14.510887
4      -23.239290
          ...    
1995   -10.086859
1996    32.394302
1997     7.504486
1998   -14.713305
1999   -16.362164
Name: z, Length: 2000, dtype: float32

In [138]:
Z

array([[-0.00982064, -0.01319036, -0.01754004, ..., -0.023092  ,
        -0.01754004, -0.01319036],
       [-0.01319036, -0.01771632, -0.02355849, ..., -0.03101548,
        -0.02355849, -0.01771632],
       [-0.01754004, -0.02355849, -0.03132719, ..., -0.0412432 ,
        -0.03132719, -0.02355849],
       ...,
       [-0.01373043, -0.01985735, -0.02800271, ...,  0.11812427,
         0.11827665,  0.11591304],
       [-0.01289122, -0.01801753, -0.02475215, ...,  0.04437913,
         0.04788803,  0.04940405],
       [-0.01097235, -0.01507265, -0.02042145, ...,  0.00983613,
         0.01423619,  0.01709512]])